In [7]:

%pylab inline
import catboost
import pandas as pd
import pickle
import tqdm

import seaborn as sns




def percentile_borders(series):
    return [int(series.quantile(perc)) for perc in [0.05, 0.2, 0.3, 0.4, 0.5, 0.65, 0.8, 0.95]]

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Load data, create target

In [2]:
set_name = 'set3'
path_train_set = 'taxi-idao/data/train/{}.csv'.format(set_name)

data = pd.read_csv(path_train_set)
data.datetime = data.datetime.apply(
    lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
data = data.sort_values('datetime')
data.head()

,datetime,num_orders
0,2018-02-01 00:00:00,0
1,2018-02-01 00:01:00,0
2,2018-02-01 00:02:00,0
3,2018-02-01 00:03:00,0
4,2018-02-01 00:04:00,0


In [3]:
target_positions = {
    'set1': [10, 30, 45, 60, 75],
    'set2': [5, 10, 15, 20, 25],
    'set3': [5, 7, 9, 11, 13]
}[set_name]



HOUR_IN_MINUTES = 60
DAY_IN_MINUTES = 24 * HOUR_IN_MINUTES
WEEK_IN_MINUTES = 7 * DAY_IN_MINUTES

MAX_TIME = DAY_IN_MINUTES


samples = {
    'datetime': [],
    'history': []}

for position in target_positions:
    samples['target_{}'.format(position)] = []
    
num_orders = data.num_orders.values



# start after 2 weeks because of history
# finish earlier because of target calculation
for i in range(2 * WEEK_IN_MINUTES,
               len(num_orders) - 2 * DAY_IN_MINUTES):
    
    samples['datetime'].append(data.datetime[i])
    samples['history'].append(num_orders[i-2*WEEK_IN_MINUTES:i])
    
    # cumsum not for all array because of time economy
    cumsum_num_orders = num_orders[i+1:i+1+2*DAY_IN_MINUTES].cumsum()
    for position in target_positions:
        orders_by_positions = np.where(cumsum_num_orders >= position)[0]
        if len(orders_by_positions):
            time = orders_by_positions[0] + 1
        else:
            # if no orders in last days
            time = MAX_TIME
        samples['target_{}'.format(position)].append(time)

In [5]:
df = pd.DataFrame.from_dict(samples)
df.head()

,datetime,history,target_5,target_7,target_9,target_11,target_13
0,2018-02-15 00:00:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",308,418,421,820,1167
1,2018-02-15 00:01:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",307,417,420,819,1166
2,2018-02-15 00:02:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",306,416,419,818,1165
3,2018-02-15 00:03:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",305,415,418,817,1164
4,2018-02-15 00:04:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",304,414,417,816,1163


## Base features

In [6]:
df['weekday'] = df.datetime.apply(lambda x: x.weekday())
df['hour'] = df.datetime.apply(lambda x: x.hour)
df['minute'] = df.datetime.apply(lambda x: x.minute)


SHIFTS = [
    HOUR_IN_MINUTES // 4,
    HOUR_IN_MINUTES // 2,
    HOUR_IN_MINUTES,
    DAY_IN_MINUTES,
    DAY_IN_MINUTES * 2,
    WEEK_IN_MINUTES,
    WEEK_IN_MINUTES * 2]
WINDOWS = [
    HOUR_IN_MINUTES // 4 - 1,
    HOUR_IN_MINUTES // 2 - 1,
    HOUR_IN_MINUTES - 1,
    DAY_IN_MINUTES - 1,
    DAY_IN_MINUTES * 2 - 1,
    WEEK_IN_MINUTES - 1,
    WEEK_IN_MINUTES * 2 - 1]


all_features_names = ['weekday', 'hour', 'minute']
for shift in SHIFTS:
    for window in WINDOWS:
        if window > shift:
            continue
        all_features_names.append('num_orders_{}_{}'.format(shift, window))
        df['num_orders_{}_{}'.format(shift, window)] = \
            df.history.apply(lambda x: x[-shift : -shift + window].sum())

## Percentiles

In [8]:
windows = {}
for tg in target_positions:
    windows[tg] = percentile_borders(df['target_' + str(tg)])
windows_names = ['5%', '20%', '30%', '40%', '50%', '65%', '80%', '95%']
windows

{5: [56, 123, 158, 195, 234, 305, 408, 642],
 7: [95, 185, 232, 278, 328, 410, 540, 819],
 9: [138, 249, 306, 362, 420, 516, 670, 989],
 11: [179, 315, 382, 446, 512, 630, 797, 1139],
 13: [235, 379, 456, 533, 605, 737, 925, 1297]}

In [9]:
SHIFTS_PERCENTILE = [WEEK_IN_MINUTES, WEEK_IN_MINUTES * 2, DAY_IN_MINUTES]
WINDOWS_PERCENTILE = {5: [56, 123, 158, 195, 234, 305, 408, 642],
                      7: [95, 185, 232, 278, 328, 410, 540, 819],
                      9: [138, 249, 306, 362, 420, 516, 670, 989],
                      11: [179, 315, 382, 446, 512, 630, 797, 1139],
                      13: [235, 379, 456, 533, 605, 737, 925, 1297]}


shifts_names = ['week_ago', 'two_weeks_ago', 'day_ago']
windows_names = ['5%', '20%', '30%', '40%', '50%', '65%', '80%', '95%']

In [10]:
### part 1

In [11]:
df_history = df.history.values

for target in tqdm.tqdm(target_positions):
    for i, shift in enumerate(SHIFTS_PERCENTILE):
        for j, window in enumerate(WINDOWS_PERCENTILE[target]):
            df['num_orders_target_{}_{}_{}'.format(target, shifts_names[i], windows_names[j])] = list(map(lambda x: x[-shift: -shift + window].sum(), df_history))

100%|██████████| 5/5 [02:10<00:00, 26.06s/it]


In [12]:
### part 2

In [13]:
for target in tqdm.tqdm(target_positions):
    for j, window in enumerate(WINDOWS_PERCENTILE[target]):
        col_name = 'num_orders_target_{}_previous_{}_perc'.format(target, windows_names[j])
        df[col_name] = list(map(lambda x: x[-window:].sum(), df_history))

100%|██████████| 5/5 [00:43<00:00,  8.62s/it]


In [15]:
df.head(2)

,datetime,history,target_5,target_7,target_9,target_11,target_13,weekday,hour,minute,...,num_orders_target_11_previous_80%_perc,num_orders_target_11_previous_95%_perc,num_orders_target_13_previous_5%_perc,num_orders_target_13_previous_20%_perc,num_orders_target_13_previous_30%_perc,num_orders_target_13_previous_40%_perc,num_orders_target_13_previous_50%_perc,num_orders_target_13_previous_65%_perc,num_orders_target_13_previous_80%_perc,num_orders_target_13_previous_95%_perc
0,2018-02-15 00:00:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",308,418,421,820,1167,3,0,0,...,11,15,6,8,9,10,10,11,13,18
1,2018-02-15 00:01:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",307,417,420,819,1166,3,0,1,...,11,15,6,8,9,10,10,11,13,18


## Creating lists

In [22]:
ALL_FEATURES = df.columns[7:]

In [23]:
len(ALL_FEATURES)

191

In [24]:
ALL_FEATURES[:31]

Index(['weekday', 'hour', 'minute', 'num_orders_15_14', 'num_orders_30_14',
       'num_orders_30_29', 'num_orders_60_14', 'num_orders_60_29',
       'num_orders_60_59', 'num_orders_1440_14', 'num_orders_1440_29',
       'num_orders_1440_59', 'num_orders_1440_1439', 'num_orders_2880_14',
       'num_orders_2880_29', 'num_orders_2880_59', 'num_orders_2880_1439',
       'num_orders_2880_2879', 'num_orders_10080_14', 'num_orders_10080_29',
       'num_orders_10080_59', 'num_orders_10080_1439', 'num_orders_10080_2879',
       'num_orders_10080_10079', 'num_orders_20160_14', 'num_orders_20160_29',
       'num_orders_20160_59', 'num_orders_20160_1439', 'num_orders_20160_2879',
       'num_orders_20160_10079', 'num_orders_20160_20159'],
      dtype='object')

In [26]:
TARGET_FEATURES_POSITIONS = {}
TARGET_FEATURES_POSITIONS[5] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[7] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[9] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[11] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[13] = [i for i in range(31)]

In [27]:
### target 10
TARGET_FEATURES_POSITIONS[5] = TARGET_FEATURES_POSITIONS[5] + [i for i in range(31,31+24)]
TARGET_FEATURES_POSITIONS[7] = TARGET_FEATURES_POSITIONS[7] + [i for i in range(31+24,31+24*2)]
TARGET_FEATURES_POSITIONS[9] = TARGET_FEATURES_POSITIONS[9] + [i for i in range(31+24*2,31+24*3)]
TARGET_FEATURES_POSITIONS[11] = TARGET_FEATURES_POSITIONS[11] + [i for i in range(31+24*3,31+24*4)]
TARGET_FEATURES_POSITIONS[13] = TARGET_FEATURES_POSITIONS[13] + [i for i in range(31+24*4,31+24*5)]




TARGET_FEATURES_POSITIONS[5] = TARGET_FEATURES_POSITIONS[5] + [i for i in range(151,151+8)]
TARGET_FEATURES_POSITIONS[7] = TARGET_FEATURES_POSITIONS[7] + [i for i in range(151+8,151+8*2)]
TARGET_FEATURES_POSITIONS[9] = TARGET_FEATURES_POSITIONS[9] + [i for i in range(151+8*2,151+8*3)]
TARGET_FEATURES_POSITIONS[11] = TARGET_FEATURES_POSITIONS[11] + [i for i in range(151+8*3,151+8*4)]
TARGET_FEATURES_POSITIONS[13] = TARGET_FEATURES_POSITIONS[13] + [i for i in range(151+8*4,151+8*5)]

# Split data

In [34]:
df.datetime.min(), df.datetime.max()
df_train = df.loc[df.datetime <= df.datetime.max() - datetime.timedelta(days=28)]
df_test = df.loc[(df.datetime > df.datetime.max() - datetime.timedelta(days=14))]

In [35]:
target_cols = ['target_{}'.format(position) for position in target_positions]

y_train = df_train[target_cols]
y_test = df_test[target_cols]

df_train = df_train.drop(['datetime', 'history'] + target_cols, axis=1)
df_test = df_test.drop(['datetime', 'history'] + target_cols, axis=1)



def sMAPE(y_true, y_predict, shift=0):
    return 2 * np.mean(
        np.abs(y_true - y_predict) /
        (np.abs(y_true) + np.abs(y_predict) + shift))


model_to_save = {
    'models': {}
}

# Train for validation

In [37]:
### all features
for position in target_positions:
    model = catboost.CatBoostRegressor(
        iterations=1500, learning_rate=1.0, loss_function='MAE')
    model.fit(
        X=df_train[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]],
        y=y_train['target_{}'.format(position)],
        use_best_model=True,
        eval_set=(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]], y_test['target_{}'.format(position)]),
        verbose=True)
    y_predict = model.predict(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]])
    
    print('target_{}'.format(position))
    print('stupid:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_train['target_{}'.format(position)].median())))
    print('model:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_predict)))
    print()
    
    model_to_save['models'][position] = model

0:	learn: 297.4977849	test: 175.4389537	best: 175.4389537 (0)	total: 73.2ms	remaining: 1m 49s
1:	learn: 296.9978001	test: 174.9389689	best: 174.9389689 (1)	total: 145ms	remaining: 1m 48s
2:	learn: 296.4978153	test: 174.4389841	best: 174.4389841 (2)	total: 207ms	remaining: 1m 43s
3:	learn: 295.9978305	test: 173.9389993	best: 173.9389993 (3)	total: 267ms	remaining: 1m 39s
4:	learn: 295.4978457	test: 173.4390145	best: 173.4390145 (4)	total: 428ms	remaining: 2m 7s
5:	learn: 294.9978609	test: 172.9390297	best: 172.9390297 (5)	total: 504ms	remaining: 2m 5s
6:	learn: 294.4978761	test: 172.4390449	best: 172.4390449 (6)	total: 566ms	remaining: 2m
7:	learn: 293.9978913	test: 171.9390601	best: 171.9390601 (7)	total: 624ms	remaining: 1m 56s
8:	learn: 293.4979268	test: 171.4391249	best: 171.4391249 (8)	total: 729ms	remaining: 2m
9:	learn: 292.9980947	test: 170.9393344	best: 170.9393344 (9)	total: 890ms	remaining: 2m 12s
10:	learn: 292.4982495	test: 170.4405516	best: 170.4405516 (10)	total: 1.06s	re

87:	learn: 255.2178749	test: 135.8999016	best: 135.8999016 (87)	total: 14.8s	remaining: 3m 57s
88:	learn: 254.7617621	test: 135.4997226	best: 135.4997226 (88)	total: 14.9s	remaining: 3m 56s
89:	learn: 254.3067000	test: 135.1006392	best: 135.1006392 (89)	total: 15.1s	remaining: 3m 56s
90:	learn: 253.8525075	test: 134.6940305	best: 134.6940305 (90)	total: 15.3s	remaining: 3m 56s
91:	learn: 253.3992458	test: 134.2965651	best: 134.2965651 (91)	total: 15.5s	remaining: 3m 56s
92:	learn: 252.9468652	test: 133.9477786	best: 133.9477786 (92)	total: 15.6s	remaining: 3m 56s
93:	learn: 252.4956832	test: 133.5635714	best: 133.5635714 (93)	total: 15.8s	remaining: 3m 55s
94:	learn: 252.0453465	test: 133.1700442	best: 133.1700442 (94)	total: 15.9s	remaining: 3m 55s
95:	learn: 251.5960541	test: 132.7887153	best: 132.7887153 (95)	total: 16.1s	remaining: 3m 55s
96:	learn: 251.1472895	test: 132.3971685	best: 132.3971685 (96)	total: 16.3s	remaining: 3m 55s
97:	learn: 250.6994054	test: 132.0835052	best: 132

172:	learn: 220.0540983	test: 108.7253980	best: 108.7253980 (172)	total: 30s	remaining: 3m 50s
173:	learn: 219.6860871	test: 108.4852452	best: 108.4852452 (173)	total: 30.2s	remaining: 3m 49s
174:	learn: 219.3194138	test: 108.2255745	best: 108.2255745 (174)	total: 30.4s	remaining: 3m 50s
175:	learn: 218.9530050	test: 107.9572454	best: 107.9572454 (175)	total: 30.5s	remaining: 3m 49s
176:	learn: 218.5883358	test: 107.7022857	best: 107.7022857 (176)	total: 30.8s	remaining: 3m 50s
177:	learn: 218.2241400	test: 107.4370304	best: 107.4370304 (177)	total: 31s	remaining: 3m 50s
178:	learn: 217.8616664	test: 107.1809331	best: 107.1809331 (178)	total: 31.2s	remaining: 3m 49s
179:	learn: 217.5002422	test: 106.9159457	best: 106.9159457 (179)	total: 31.4s	remaining: 3m 50s
180:	learn: 217.1399550	test: 106.6668966	best: 106.6668966 (180)	total: 31.6s	remaining: 3m 50s
181:	learn: 216.7808897	test: 106.4275329	best: 106.4275329 (181)	total: 31.8s	remaining: 3m 50s
182:	learn: 216.4225106	test: 106.

258:	learn: 192.3195075	test: 91.1571098	best: 91.1571098 (258)	total: 47.7s	remaining: 3m 48s
259:	learn: 192.0419939	test: 90.9983073	best: 90.9983073 (259)	total: 47.8s	remaining: 3m 48s
260:	learn: 191.7655811	test: 90.8318966	best: 90.8318966 (260)	total: 48s	remaining: 3m 47s
261:	learn: 191.4899078	test: 90.6720119	best: 90.6720119 (261)	total: 48.2s	remaining: 3m 47s
262:	learn: 191.2152029	test: 90.5129500	best: 90.5129500 (262)	total: 48.4s	remaining: 3m 47s
263:	learn: 190.9402996	test: 90.3924972	best: 90.3924972 (263)	total: 48.6s	remaining: 3m 47s
264:	learn: 190.6664678	test: 90.2729327	best: 90.2729327 (264)	total: 48.8s	remaining: 3m 47s
265:	learn: 190.3948723	test: 90.1207886	best: 90.1207886 (265)	total: 49s	remaining: 3m 47s
266:	learn: 190.1241491	test: 89.9780837	best: 89.9780837 (266)	total: 49.1s	remaining: 3m 46s
267:	learn: 189.8540585	test: 89.8329008	best: 89.8329008 (267)	total: 49.4s	remaining: 3m 46s
268:	learn: 189.5852935	test: 89.6780125	best: 89.6780

345:	learn: 171.5141350	test: 81.0029044	best: 81.0029044 (345)	total: 1m 4s	remaining: 3m 33s
346:	learn: 171.3098707	test: 80.9213338	best: 80.9213338 (346)	total: 1m 4s	remaining: 3m 33s
347:	learn: 171.1080512	test: 80.8405047	best: 80.8405047 (347)	total: 1m 4s	remaining: 3m 33s
348:	learn: 170.9063658	test: 80.7580946	best: 80.7580946 (348)	total: 1m 4s	remaining: 3m 34s
349:	learn: 170.7051650	test: 80.6777271	best: 80.6777271 (349)	total: 1m 5s	remaining: 3m 34s
350:	learn: 170.5067437	test: 80.5965387	best: 80.5965387 (350)	total: 1m 5s	remaining: 3m 34s
351:	learn: 170.3072874	test: 80.5197656	best: 80.5197656 (351)	total: 1m 5s	remaining: 3m 34s
352:	learn: 170.1084154	test: 80.4619094	best: 80.4619094 (352)	total: 1m 5s	remaining: 3m 33s
353:	learn: 169.9103036	test: 80.3834008	best: 80.3834008 (353)	total: 1m 6s	remaining: 3m 33s
354:	learn: 169.7138755	test: 80.3058132	best: 80.3058132 (354)	total: 1m 6s	remaining: 3m 34s
355:	learn: 169.5170209	test: 80.2223782	best: 80.

...

1164:	learn: 274.3219884	test: 122.2241871	best: 122.2241871 (1164)	total: 3m 53s	remaining: 1m 7s
1165:	learn: 274.1449047	test: 122.2066715	best: 122.2066715 (1165)	total: 3m 53s	remaining: 1m 6s
1166:	learn: 273.9656528	test: 122.1980872	best: 122.1980872 (1166)	total: 3m 53s	remaining: 1m 6s
1167:	learn: 273.7923242	test: 122.1953703	best: 122.1953703 (1167)	total: 3m 53s	remaining: 1m 6s
1168:	learn: 273.6189413	test: 122.1682436	best: 122.1682436 (1168)	total: 3m 53s	remaining: 1m 6s
1169:	learn: 273.4469664	test: 122.1474596	best: 122.1474596 (1169)	total: 3m 53s	remaining: 1m 5s
1170:	learn: 273.2735239	test: 122.1230953	best: 122.1230953 (1170)	total: 3m 54s	remaining: 1m 5s
1171:	learn: 273.1005057	test: 122.0989615	best: 122.0989615 (1171)	total: 3m 54s	remaining: 1m 5s
1172:	learn: 272.9296393	test: 122.0785292	best: 122.0785292 (1172)	total: 3m 54s	remaining: 1m 5s
1173:	learn: 272.7579623	test: 122.0674476	best: 122.0674476 (1173)	total: 3m 54s	remaining: 1m 5s
1174:	lear

1248:	learn: 260.6615169	test: 121.1518776	best: 121.1518776 (1248)	total: 4m 8s	remaining: 49.8s
1249:	learn: 260.5082370	test: 121.1362923	best: 121.1362923 (1249)	total: 4m 8s	remaining: 49.7s
1250:	learn: 260.3594434	test: 121.1329177	best: 121.1329177 (1250)	total: 4m 8s	remaining: 49.4s
1251:	learn: 260.2062201	test: 121.1184140	best: 121.1184140 (1251)	total: 4m 8s	remaining: 49.2s
1252:	learn: 260.0543998	test: 121.1184947	best: 121.1184140 (1251)	total: 4m 8s	remaining: 49s
1253:	learn: 259.9020879	test: 121.1094695	best: 121.1094695 (1253)	total: 4m 8s	remaining: 48.8s
1254:	learn: 259.7512317	test: 121.0951459	best: 121.0951459 (1254)	total: 4m 9s	remaining: 48.6s
1255:	learn: 259.6002808	test: 121.0699503	best: 121.0699503 (1255)	total: 4m 9s	remaining: 48.4s
1256:	learn: 259.4511339	test: 121.0438559	best: 121.0438559 (1256)	total: 4m 9s	remaining: 48.2s
1257:	learn: 259.2980837	test: 121.0194621	best: 121.0194621 (1257)	total: 4m 9s	remaining: 48s
1258:	learn: 259.1522905

1332:	learn: 248.8370650	test: 120.2470035	best: 120.2415566 (1329)	total: 4m 22s	remaining: 32.9s
1333:	learn: 248.7006966	test: 120.2380650	best: 120.2380650 (1333)	total: 4m 23s	remaining: 32.7s
1334:	learn: 248.5733480	test: 120.2272332	best: 120.2272332 (1334)	total: 4m 23s	remaining: 32.6s
1335:	learn: 248.4446540	test: 120.2135165	best: 120.2135165 (1335)	total: 4m 23s	remaining: 32.4s
1336:	learn: 248.3178120	test: 120.1997727	best: 120.1997727 (1336)	total: 4m 23s	remaining: 32.2s
1337:	learn: 248.1883071	test: 120.1794873	best: 120.1794873 (1337)	total: 4m 23s	remaining: 32s
1338:	learn: 248.0618798	test: 120.1625049	best: 120.1625049 (1338)	total: 4m 24s	remaining: 31.8s
1339:	learn: 247.9358003	test: 120.1456867	best: 120.1456867 (1339)	total: 4m 24s	remaining: 31.6s
1340:	learn: 247.8031086	test: 120.1385686	best: 120.1385686 (1340)	total: 4m 24s	remaining: 31.4s
1341:	learn: 247.6686984	test: 120.1300413	best: 120.1300413 (1341)	total: 4m 24s	remaining: 31.2s
1342:	learn:

1416:	learn: 238.6813101	test: 119.9093325	best: 119.8679907 (1407)	total: 4m 38s	remaining: 16.3s
1417:	learn: 238.5674991	test: 119.9073657	best: 119.8679907 (1407)	total: 4m 38s	remaining: 16.1s
1418:	learn: 238.4548173	test: 119.9073327	best: 119.8679907 (1407)	total: 4m 38s	remaining: 15.9s
1419:	learn: 238.3413055	test: 119.9103027	best: 119.8679907 (1407)	total: 4m 39s	remaining: 15.7s
1420:	learn: 238.2332685	test: 119.9126823	best: 119.8679907 (1407)	total: 4m 39s	remaining: 15.5s
1421:	learn: 238.1229232	test: 119.9035497	best: 119.8679907 (1407)	total: 4m 39s	remaining: 15.3s
1422:	learn: 238.0143153	test: 119.8898615	best: 119.8679907 (1407)	total: 4m 39s	remaining: 15.1s
1423:	learn: 237.9049019	test: 119.8926923	best: 119.8679907 (1407)	total: 4m 39s	remaining: 14.9s
1424:	learn: 237.7994515	test: 119.9210103	best: 119.8679907 (1407)	total: 4m 39s	remaining: 14.7s
1425:	learn: 237.6915050	test: 119.9157297	best: 119.8679907 (1407)	total: 4m 40s	remaining: 14.5s
1426:	lear

1499:	learn: 230.1788882	test: 119.8575299	best: 119.8322574 (1487)	total: 4m 53s	remaining: 0us

bestTest = 119.8322574
bestIteration = 1487

Shrink model to first 1488 iterations.
target_13
stupid:	0.5394143681443782
model:	0.2987056833191235



# Full learning

In [40]:
df_full_train = df.drop(['datetime', 'history'] + target_cols, axis=1)
y_full_train = df[target_cols]

In [42]:
dict_iterations = {5: 1125, 
                   7: 878, 
                   9: 1450, 
                   11: 1419, 
                   13: 1450}

In [43]:
for position in target_positions:
    model = catboost.CatBoostRegressor(
        iterations=dict_iterations[position], learning_rate=1.0, loss_function='MAE')
    model.fit(
        X=df_full_train[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]],
        y=y_full_train['target_{}'.format(position)],
        use_best_model=False,
        eval_set=(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]], y_test['target_{}'.format(position)]),
        verbose=True)
    y_predict = model.predict(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]])
    
    print('target_{}'.format(position))
    print('stupid:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_train['target_{}'.format(position)].median())))
    print('model:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_predict)))
    print()
    
    model_to_save['models'][position] = model

0:	learn: 275.9141204	test: 175.4389511	best: 175.4389511 (0)	total: 43.2ms	remaining: 48.5s
1:	learn: 275.4141330	test: 174.9389637	best: 174.9389637 (1)	total: 83.5ms	remaining: 46.9s
2:	learn: 274.9141456	test: 174.4389763	best: 174.4389763 (2)	total: 141ms	remaining: 52.7s
3:	learn: 274.4141582	test: 173.9389889	best: 173.9389889 (3)	total: 201ms	remaining: 56.2s
4:	learn: 273.9141709	test: 173.4390016	best: 173.4390016 (4)	total: 258ms	remaining: 57.7s
5:	learn: 273.4141835	test: 172.9390142	best: 172.9390142 (5)	total: 320ms	remaining: 59.7s
6:	learn: 272.9141961	test: 172.4390268	best: 172.4390268 (6)	total: 366ms	remaining: 58.4s
7:	learn: 272.4142088	test: 171.9390394	best: 171.9390394 (7)	total: 407ms	remaining: 56.9s
8:	learn: 271.9142424	test: 171.4391017	best: 171.4391017 (8)	total: 449ms	remaining: 55.6s
9:	learn: 271.4143222	test: 170.9392406	best: 170.9392406 (9)	total: 557ms	remaining: 1m 2s
10:	learn: 270.9145723	test: 170.4397061	best: 170.4397061 (10)	total: 669ms	r

89:	learn: 233.0448720	test: 133.7905175	best: 133.7905175 (89)	total: 10.7s	remaining: 2m 3s
90:	learn: 232.6010815	test: 133.3908831	best: 133.3908831 (90)	total: 10.8s	remaining: 2m 2s
91:	learn: 232.1585696	test: 132.9893015	best: 132.9893015 (91)	total: 10.9s	remaining: 2m 2s
92:	learn: 231.7170888	test: 132.5920622	best: 132.5920622 (92)	total: 11s	remaining: 2m 2s
93:	learn: 231.2766680	test: 132.1941296	best: 132.1941296 (93)	total: 11.2s	remaining: 2m 2s
94:	learn: 230.8376637	test: 131.8023538	best: 131.8023538 (94)	total: 11.3s	remaining: 2m 2s
95:	learn: 230.3991136	test: 131.4125182	best: 131.4125182 (95)	total: 11.4s	remaining: 2m 2s
96:	learn: 229.9623780	test: 131.0136324	best: 131.0136324 (96)	total: 11.5s	remaining: 2m 2s
97:	learn: 229.5263948	test: 130.6251674	best: 130.6251674 (97)	total: 11.6s	remaining: 2m 1s
98:	learn: 229.0915923	test: 130.2350279	best: 130.2350279 (98)	total: 11.7s	remaining: 2m 1s
99:	learn: 228.6577755	test: 129.8565413	best: 129.8565413 (99

176:	learn: 198.8361865	test: 106.1088958	best: 106.1088958 (176)	total: 20.6s	remaining: 1m 50s
177:	learn: 198.4949147	test: 105.8648702	best: 105.8648702 (177)	total: 20.7s	remaining: 1m 50s
178:	learn: 198.1547211	test: 105.6222951	best: 105.6222951 (178)	total: 20.8s	remaining: 1m 50s
179:	learn: 197.8165810	test: 105.3823593	best: 105.3823593 (179)	total: 20.9s	remaining: 1m 49s
180:	learn: 197.4792705	test: 105.1470007	best: 105.1470007 (180)	total: 21s	remaining: 1m 49s
181:	learn: 197.1426917	test: 104.9075399	best: 104.9075399 (181)	total: 21.2s	remaining: 1m 49s
182:	learn: 196.8073981	test: 104.6647117	best: 104.6647117 (182)	total: 21.3s	remaining: 1m 49s
183:	learn: 196.4736122	test: 104.4311583	best: 104.4311583 (183)	total: 21.4s	remaining: 1m 49s
184:	learn: 196.1408326	test: 104.2049597	best: 104.2049597 (184)	total: 21.5s	remaining: 1m 49s
185:	learn: 195.8089244	test: 103.9639055	best: 103.9639055 (185)	total: 21.6s	remaining: 1m 49s
186:	learn: 195.4783481	test: 10

262:	learn: 173.6123198	test: 89.7319839	best: 89.7319839 (262)	total: 30.9s	remaining: 1m 41s
263:	learn: 173.3648388	test: 89.5822938	best: 89.5822938 (263)	total: 31.1s	remaining: 1m 41s
264:	learn: 173.1166518	test: 89.4341493	best: 89.4341493 (264)	total: 31.2s	remaining: 1m 41s
265:	learn: 172.8703866	test: 89.2832656	best: 89.2832656 (265)	total: 31.3s	remaining: 1m 41s
266:	learn: 172.6252623	test: 89.1552726	best: 89.1552726 (266)	total: 31.4s	remaining: 1m 40s
267:	learn: 172.3803714	test: 89.0117334	best: 89.0117334 (267)	total: 31.5s	remaining: 1m 40s
268:	learn: 172.1365926	test: 88.8744961	best: 88.8744961 (268)	total: 31.6s	remaining: 1m 40s
269:	learn: 171.8935779	test: 88.7409582	best: 88.7409582 (269)	total: 31.8s	remaining: 1m 40s
270:	learn: 171.6515307	test: 88.5965136	best: 88.5965136 (270)	total: 31.9s	remaining: 1m 40s
271:	learn: 171.4107399	test: 88.4507594	best: 88.4507594 (271)	total: 32.2s	remaining: 1m 40s
272:	learn: 171.1708078	test: 88.3095504	best: 88.

349:	learn: 155.2192389	test: 80.0709567	best: 80.0709567 (349)	total: 41.3s	remaining: 1m 31s
350:	learn: 155.0424748	test: 79.9867192	best: 79.9867192 (350)	total: 41.4s	remaining: 1m 31s
351:	learn: 154.8666403	test: 79.8965509	best: 79.8965509 (351)	total: 41.5s	remaining: 1m 31s
352:	learn: 154.6910190	test: 79.8175325	best: 79.8175325 (352)	total: 41.6s	remaining: 1m 31s
353:	learn: 154.5162564	test: 79.7409926	best: 79.7409926 (353)	total: 41.7s	remaining: 1m 30s
354:	learn: 154.3422475	test: 79.6615737	best: 79.6615737 (354)	total: 41.8s	remaining: 1m 30s
355:	learn: 154.1696056	test: 79.5899063	best: 79.5899063 (355)	total: 41.9s	remaining: 1m 30s
356:	learn: 153.9971129	test: 79.5120333	best: 79.5120333 (356)	total: 42.1s	remaining: 1m 30s
357:	learn: 153.8251628	test: 79.4373726	best: 79.4373726 (357)	total: 42.2s	remaining: 1m 30s
358:	learn: 153.6541862	test: 79.3548906	best: 79.3548906 (358)	total: 42.3s	remaining: 1m 30s
359:	learn: 153.4834695	test: 79.2801897	best: 79.

...

1053:	learn: 266.5308930	test: 110.2696314	best: 110.2696314 (1053)	total: 3m 57s	remaining: 1m 29s
1054:	learn: 266.3525795	test: 110.2476243	best: 110.2476243 (1054)	total: 3m 58s	remaining: 1m 29s
1055:	learn: 266.1754467	test: 110.2271207	best: 110.2271207 (1055)	total: 3m 58s	remaining: 1m 29s
1056:	learn: 266.0018899	test: 110.1971730	best: 110.1971730 (1056)	total: 3m 58s	remaining: 1m 28s
1057:	learn: 265.8232274	test: 110.1745516	best: 110.1745516 (1057)	total: 3m 59s	remaining: 1m 28s
1058:	learn: 265.6461825	test: 110.1528686	best: 110.1528686 (1058)	total: 3m 59s	remaining: 1m 28s
1059:	learn: 265.4735973	test: 110.1232356	best: 110.1232356 (1059)	total: 3m 59s	remaining: 1m 28s
1060:	learn: 265.2981122	test: 110.0697313	best: 110.0697313 (1060)	total: 3m 59s	remaining: 1m 27s
1061:	learn: 265.1230941	test: 110.0475687	best: 110.0475687 (1061)	total: 4m	remaining: 1m 27s
1062:	learn: 264.9474476	test: 110.0261837	best: 110.0261837 (1062)	total: 4m	remaining: 1m 27s
1063:	le

1136:	learn: 252.7398690	test: 108.3864542	best: 108.3864542 (1136)	total: 4m 17s	remaining: 1m 10s
1137:	learn: 252.5844725	test: 108.3615160	best: 108.3615160 (1137)	total: 4m 17s	remaining: 1m 10s
1138:	learn: 252.4319181	test: 108.3414832	best: 108.3414832 (1138)	total: 4m 17s	remaining: 1m 10s
1139:	learn: 252.2784674	test: 108.3260479	best: 108.3260479 (1139)	total: 4m 17s	remaining: 1m 10s
1140:	learn: 252.1232938	test: 108.3163403	best: 108.3163403 (1140)	total: 4m 18s	remaining: 1m 9s
1141:	learn: 251.9710086	test: 108.2874330	best: 108.2874330 (1141)	total: 4m 18s	remaining: 1m 9s
1142:	learn: 251.8183361	test: 108.2603465	best: 108.2603465 (1142)	total: 4m 18s	remaining: 1m 9s
1143:	learn: 251.6622830	test: 108.2522682	best: 108.2522682 (1143)	total: 4m 18s	remaining: 1m 9s
1144:	learn: 251.5062796	test: 108.2369005	best: 108.2369005 (1144)	total: 4m 18s	remaining: 1m 8s
1145:	learn: 251.3510926	test: 108.2222092	best: 108.2222092 (1145)	total: 4m 19s	remaining: 1m 8s
1146:	

1219:	learn: 240.7552371	test: 107.1015076	best: 107.1015076 (1219)	total: 4m 38s	remaining: 52.6s
1220:	learn: 240.6229244	test: 107.0851634	best: 107.0851634 (1220)	total: 4m 39s	remaining: 52.4s
1221:	learn: 240.4861032	test: 107.0805031	best: 107.0805031 (1221)	total: 4m 39s	remaining: 52.1s
1222:	learn: 240.3540223	test: 107.0768278	best: 107.0768278 (1222)	total: 4m 39s	remaining: 51.9s
1223:	learn: 240.2223551	test: 107.0549733	best: 107.0549733 (1223)	total: 4m 40s	remaining: 51.8s
1224:	learn: 240.0909293	test: 107.0409279	best: 107.0409279 (1224)	total: 4m 41s	remaining: 51.6s
1225:	learn: 239.9592726	test: 107.0221574	best: 107.0221574 (1225)	total: 4m 41s	remaining: 51.5s
1226:	learn: 239.8273661	test: 107.0001453	best: 107.0001453 (1226)	total: 4m 42s	remaining: 51.3s
1227:	learn: 239.6945202	test: 106.9959724	best: 106.9959724 (1227)	total: 4m 43s	remaining: 51.2s
1228:	learn: 239.5589007	test: 106.9915770	best: 106.9915770 (1228)	total: 4m 43s	remaining: 51s
1229:	learn:

1303:	learn: 230.3519270	test: 105.7809439	best: 105.7809439 (1303)	total: 5m 14s	remaining: 35.2s
1304:	learn: 230.2401795	test: 105.7680100	best: 105.7680100 (1304)	total: 5m 14s	remaining: 35s
1305:	learn: 230.1267470	test: 105.7493448	best: 105.7493448 (1305)	total: 5m 15s	remaining: 34.7s
1306:	learn: 230.0146099	test: 105.7308017	best: 105.7308017 (1306)	total: 5m 15s	remaining: 34.5s
1307:	learn: 229.9022558	test: 105.7187604	best: 105.7187604 (1307)	total: 5m 15s	remaining: 34.3s
1308:	learn: 229.7914787	test: 105.7059788	best: 105.7059788 (1308)	total: 5m 16s	remaining: 34.1s
1309:	learn: 229.6810494	test: 105.6933308	best: 105.6933308 (1309)	total: 5m 16s	remaining: 33.8s
1310:	learn: 229.5707869	test: 105.6831098	best: 105.6831098 (1310)	total: 5m 16s	remaining: 33.6s
1311:	learn: 229.4585539	test: 105.6726377	best: 105.6726377 (1311)	total: 5m 16s	remaining: 33.3s
1312:	learn: 229.3441181	test: 105.6482624	best: 105.6482624 (1312)	total: 5m 17s	remaining: 33.1s
1313:	learn:

1387:	learn: 221.4927885	test: 104.6984011	best: 104.6984011 (1387)	total: 5m 37s	remaining: 15.1s
1388:	learn: 221.3954507	test: 104.6843702	best: 104.6843702 (1388)	total: 5m 38s	remaining: 14.9s
1389:	learn: 221.2948947	test: 104.6664149	best: 104.6664149 (1389)	total: 5m 38s	remaining: 14.6s
1390:	learn: 221.1986984	test: 104.6501254	best: 104.6501254 (1390)	total: 5m 38s	remaining: 14.4s
1391:	learn: 221.1006686	test: 104.6469542	best: 104.6469542 (1391)	total: 5m 38s	remaining: 14.1s
1392:	learn: 221.0043592	test: 104.6291872	best: 104.6291872 (1392)	total: 5m 39s	remaining: 13.9s
1393:	learn: 220.9041006	test: 104.6231053	best: 104.6231053 (1393)	total: 5m 39s	remaining: 13.6s
1394:	learn: 220.8084508	test: 104.6174500	best: 104.6174500 (1394)	total: 5m 39s	remaining: 13.4s
1395:	learn: 220.7106572	test: 104.6028918	best: 104.6028918 (1395)	total: 5m 39s	remaining: 13.1s
1396:	learn: 220.6111527	test: 104.5969689	best: 104.5969689 (1396)	total: 5m 39s	remaining: 12.9s
1397:	lear

In [44]:
pickle.dump(model_to_save, open('set3_features_model_full_train.pkl', 'wb'))